In [1]:
import os
import openai
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)

openai.organization = os.getenv('OPENAI_API_ORG')
openai.api_key  = os.getenv('OPENAI_API_KEY')

In [2]:
def listmodels(containing="gpt"):
    print([d.id for d in openai.Model.list().data if containing in d.id])

In [3]:
listmodels()

['gpt-3.5-turbo-16k-0613', 'gpt-3.5-turbo', 'gpt-3.5-turbo-16k', 'gpt-3.5-turbo-0613', 'gpt-3.5-turbo-instruct-0914', 'gpt-3.5-turbo-0301', 'gpt-3.5-turbo-instruct']


In [4]:
@retry(wait=wait_random_exponential(multiplier=1, min=4, max=10), stop=stop_after_attempt(6))
def get_completion(prompt, model="gpt-3.5-turbo-16k"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]


# Prompting Principles

## Principle 1: Write clear and specific instructions

### Tactic 1: Use delimiters to clearly indicate distinct parts of the input

Delimiters can be anything like: ```, """, < >, <tag> </tag>, :

In [5]:
text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""

In [6]:
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
```{text}```
"""
print(get_completion(prompt))

To guide a model towards the desired output and reduce irrelevant or incorrect responses, it is important to provide clear and specific instructions, which can be achieved through longer prompts that offer more clarity and context.


In [7]:
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence with less than 40 words.
```{text}```
"""
print(get_completion(prompt))

To guide a model towards desired output and reduce irrelevant responses, provide clear and specific instructions, which can be longer for more clarity and context.


### Tactic 2: look for structure output

In [8]:
prompt = f"""
Generate a list of three  movie films titles along \ 
with their authors and genres that can be considered by film experts as the best in history 
Provide them in JSON format with the following keys: 
film_id, title, director, genre and year of release. 
"""
response = get_completion(prompt)
print(response)

{
  "films": [
    {
      "film_id": 1,
      "title": "The Godfather",
      "director": "Francis Ford Coppola",
      "genre": "Crime, Drama",
      "year_of_release": 1972
    },
    {
      "film_id": 2,
      "title": "Citizen Kane",
      "director": "Orson Welles",
      "genre": "Drama, Mystery",
      "year_of_release": 1941
    },
    {
      "film_id": 3,
      "title": "The Shawshank Redemption",
      "director": "Frank Darabont",
      "genre": "Drama",
      "year_of_release": 1994
    }
  ]
}


### Tactic 3: Ask the model to check whether conditions are satisfied

In [9]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \ 
water boiling. While that's happening, \ 
grab a cup and put a tea bag in it. Once the water is \ 
hot enough, just pour it over the tea bag. \ 
Let it sit for a bit so the tea can steep. After a \ 
few minutes, take out the tea bag. If you \ 
like, you can add some sugar or milk to taste. \ 
And that's it! You've got yourself a delicious \ 
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Once the water is hot enough, pour it over the tea bag.
Step 4 - Let it sit for a bit so the tea can steep.
Step 5 - After a few minutes, take out the tea bag.
Step 6 - If you like, add some sugar or milk to taste.
Step 7 - Enjoy your delicious cup of tea.


In [10]:
text_2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \ 
walk in the park. The flowers are blooming, and the \ 
trees are swaying gently in the breeze. People \ 
are out and about, enjoying the lovely weather. \ 
Some are having picnics, while others are playing \ 
games or simply relaxing on the grass. It's a \ 
perfect day to spend time outdoors and appreciate the \ 
beauty of nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

Completion for Text 2:
No steps provided.


### Tactic 4: "Few-shot" prompting

In [11]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \ 
valley flows from a modest spring; the \ 
grandest symphony originates from a single note; \ 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

<grandparent>: Resilience is the unwavering strength that emerges from facing adversity. It is the ability to bounce back, to rise above challenges, and to persevere in the face of setbacks. Like a mighty oak tree that withstands the fiercest storms, resilience allows us to weather the storms of life and come out stronger on the other side.


I have tried to come up with a fragment in the style of "El Quijote", with no luck, even providing an example.

In [12]:
prompt = f"""
Your task is to anwer in a consistent style with Sancho Panza from Cervante's Quijote. \
The text should be consistent to spanish language from early XVII century. \
The text should chain some popular proverbs and it should be written in spanish, as you can read from the example:

<Sancho>: Bien sé firmar mi nombre que cuando fui prioste en mi lugar, \
aprendí a hacer unas letras como de marca de fardo, que decían que decía  \
mi nombre; cuanto más, que fingiré que tengo tullida la mano derecha, \
y haré que firme otro por mí; que para todo hay remedio, si no es para \
la muerte; y, teniendo yo el mando y el palo, haré lo que quisiere;  \
cuanto más, que el que tiene el padre alcalde… Y, siendo yo gobernador, \
que es más que ser alcalde, ¡llegaos, que la dejan ver! No, sino popen \
y calóñenme, que vendrán por lana y volverán trasquilados; y a quien Dios \
quiere bien, la casa le sabe; y las necedades del rico por sentencias pasan \
en el mundo; y, siéndolo yo, siendo gobernador y juntamente liberal, \
como lo pienso ser, no habrá falta que se me parezca. No, sino haceos miel,\
y paparos han moscas; tanto vales cuanto tienes, decía una mi agüela, \
y del hombre arraigado no te verás vengado.

Which are the main problems in life?
"""
response = get_completion(prompt)
print(response)

<Sancho>: ¡Ay, amigo mío! En esta vida, los problemas son tantos como granos de arena en la playa y tantos como estrellas en el cielo. Pero si me preguntas cuáles son los principales, te diré que son tres: la falta de dinero, la falta de salud y la falta de amor.

La falta de dinero, amigo, es como un viento fuerte que sopla sin cesar, llevándose consigo la tranquilidad y la alegría. Sin dinero, no se puede satisfacer las necesidades básicas ni disfrutar de los placeres de la vida. Como dice el refrán, "Dinero llama dinero", y es cierto que sin dinero, la vida se vuelve más difícil y llena de preocupaciones.

La falta de salud, por su parte, es como una sombra oscura que se cierne sobre nosotros. Sin salud, no se puede disfrutar de las bendiciones de la vida ni realizar las tareas diarias con facilidad. Como dice el refrán, "En la salud está la verdadera riqueza", y es cierto que sin salud, todas las riquezas del mundo no tienen valor.

Y por último, la falta de amor, amigo mío, es com

## Principle: Give the model time to “think”

### Tactic 1: Specify the steps required to complete a task

In [13]:
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
1 - Jack and Jill, siblings, go on a quest to fetch water from a hilltop well, but encounter misfortune when Jack trips on a stone and tumbles down the hill, with Jill following suit, yet they return home and remain undeterred in their adventurous spirits.

2 - Jack et Jill, frère et sœur, partent en quête d'eau d'un puits au sommet d'une colline, mais rencontrent un malheur lorsque Jack trébuche sur une pierre et dévale la colline, suivi par Jill, pourtant ils rentrent chez eux et restent déterminés dans leur esprit d'aventure.

3 - Jack, Jill

4 - {
  "french_summary": "Jack et Jill, frère et sœur, partent en quête d'eau d'un puits au sommet d'une colline, mais rencontrent un malheur lorsque Jack trébuche sur une pierre et dévale la colline, suivi par Jill, pourtant ils rentrent chez eux et restent déterminés dans leur esprit d'aventure.",
  "num_names": 2
}


In [14]:
prompt_2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the 
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in Italian summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)


Completion for prompt 2:
Summary: Jack and Jill, siblings, go on a quest to fetch water from a hilltop well but encounter misfortune along the way.

Translation: Jack et Jill, frère et sœur, partent en quête d'eau d'un puits au sommet d'une colline mais rencontrent des malheurs en chemin.

Names: Jack, Jill

Output JSON: {"french_summary": "Jack et Jill, frère et sœur, partent en quête d'eau d'un puits au sommet d'une colline mais rencontrent des malheurs en chemin.", "num_names": 2}


#### Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion

In [15]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

The student's solution is correct. They correctly identified the costs for land, solar panels, and maintenance, and calculated the total cost as a function of the number of square feet.


In [16]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

To calculate the total cost for the first year of operations, we need to add up the costs of land, solar panels, and maintenance.

Let x be the size of the installation in square feet.

Costs:
1. Land cost: $100 * x
2. Solar panel cost: $250 * x
3. Maintenance cost: $100,000 + $10 * x

Total cost: $100 * x + $250 * x + $100,000 + $10 * x = $360 * x + $100,000

Is the student's solution the same as the actual solution just calculated:
No

Student grade:
Incorrect


## Model limits: Hallucinations

In [17]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie
"""
response = get_completion(prompt)
print(response)

The AeroGlide UltraSlim Smart Toothbrush by Boie is a technologically advanced toothbrush designed to provide a superior brushing experience. Boie is a company known for its innovative oral care products, and the AeroGlide UltraSlim Smart Toothbrush is no exception.

One of the standout features of this toothbrush is its ultra-slim design. The brush head is only 2mm thick, making it much thinner than traditional toothbrushes. This slim profile allows for better access to hard-to-reach areas of the mouth, ensuring a thorough and effective clean.

The AeroGlide UltraSlim Smart Toothbrush also incorporates smart technology. It connects to a mobile app via Bluetooth, allowing users to track their brushing habits and receive personalized recommendations for improving their oral hygiene routine. The app provides real-time feedback on brushing technique, duration, and coverage, helping users to achieve optimal oral health.

The toothbrush features soft, antimicrobial bristles made from a dura

In [18]:
prompt = f"""
Tell me about Veridas Authentication Solutions for solving face authentication in primary schools
"""
response = get_completion(prompt)
print(response)

Veridas Authentication Solutions is a company that specializes in providing advanced face authentication technology for various industries, including primary schools. Their solutions aim to enhance security, streamline processes, and improve user experience.

In the context of primary schools, Veridas offers a face authentication system that can be used for various purposes, such as student attendance tracking, access control, and identity verification. Here are some key features and benefits of their solutions:

1. Face Recognition Technology: Veridas utilizes state-of-the-art face recognition algorithms that can accurately identify individuals based on their facial features. This technology ensures reliable and secure authentication, minimizing the risk of unauthorized access.

2. Attendance Tracking: Veridas' face authentication system can automate the attendance tracking process in primary schools. Students can simply stand in front of a camera, and their attendance will be recorde